In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

# Cálculo RR Brasil

In [3]:
# Base SIM_DOFET
df_sim = pd.read_csv('./base_limpa/base_sim_dofet_limpa_remocao.csv')
# Base SINASC
df_sinasc = pd.read_csv('./base_limpa/base_sinasc_limpa_remocao.csv')

In [12]:
def calcular_prevalencia_com_ic(n_total, prevalencia, nivel_confianca=0.95):
     # Calcula o erro padrão
    erro_padrao = np.sqrt((prevalencia * (1 - prevalencia)) / n_total)
    
    # Calcula o valor crítico Z para o nível de confiança
    z = norm.ppf(1 - (1 - nivel_confianca) / 2)
    
    # Calcula os limites do intervalo de confiança
    ic_inferior = prevalencia - z * erro_padrao
    ic_superior = prevalencia + z * erro_padrao
    
    return f'{round(prevalencia*100, 2)} ({round(ic_inferior*100, 2)}-{round(ic_superior*100, 2)})'

In [19]:
def calculate_rr_e_prevalencia(df_obito, df_nasc, region=None, semana=22):
    # Filtrar nascimentos e óbitos a partir da semana gestacional especificada
    df_nasc = df_nasc[df_nasc['SEMAGESTAC'] >= semana].reset_index(drop=True)
    df_obito = df_obito[df_obito['SEMAGESTAC'] >= semana].reset_index(drop=True)
    
    # Filtrar óbitos apenas da semana específica para o cálculo do RR
    df_obito_semana = df_obito[df_obito['SEMAGESTAC'] == semana]
    df_nasc_semana = df_nasc[df_nasc['SEMAGESTAC'] == semana]
    if region:
        df_nasc = df_nasc[df_nasc['nasc_REGIAO'] == region].reset_index(drop=True)
        df_obito_semana = df_obito_semana[df_obito_semana['ocor_REGIAO'] == region].reset_index(drop=True)

    # Contagem de óbitos da semana e nascimentos acumulados desde a semana
    # Substitua 'Tamanho' pelo nome correto da coluna que contém os dados
    deaths = df_obito_semana.groupby('cat_peso_calc').size()  # Contagem de óbitos por grupo
    births = df_nasc.groupby('cat_peso_calc').size()  # Contagem de nascimentos por grupo
    births_semana = df_nasc_semana.groupby('cat_peso_calc').size()  # Contagem de nascimentos por grupo
    # Estatísticas gerais
    fetos_em_risco = len(df_nasc) + len(df_obito)  # Todos os fetos a partir da semana
    nascimetos_sem = len(df_nasc_semana)  # Nascimentos da semana
    obitos_sem = len(df_obito_semana)  # Óbitos da semana
    total_nascimentos = nascimetos_sem + obitos_sem

    # Prevalencias
    prev_AIG = (births_semana['AIG'] + deaths['AIG'])/total_nascimentos
    prev_aig_ic = calcular_prevalencia_com_ic(n_total=total_nascimentos, prevalencia=prev_AIG, nivel_confianca=0.95)
    prev_PIG = (births_semana['PIG'] + deaths['PIG'])/total_nascimentos
    prev_pig_ic = calcular_prevalencia_com_ic(n_total=total_nascimentos, prevalencia=prev_PIG, nivel_confianca=0.95)
    prev_GIG =(births_semana['GIG'] + deaths['GIG'])/total_nascimentos
    prev_gig_ic = calcular_prevalencia_com_ic(n_total=total_nascimentos, prevalencia=prev_GIG, nivel_confianca=0.95)

    # Inicializar o dicionário de resultados
    rr_results = {
        'Gestational Week': semana,
        'Fetuses at Risk': fetos_em_risco,
        'Total Births': total_nascimentos,
        'Live Births': nascimetos_sem,
        'Stillbirths': obitos_sem
    }

    # Comparações
    comparisons = [
        ('AIG', 'PIG'),
        ('AIG', 'GIG'),
        ('GIG', 'PIG')
    ]
    
    for ref, exp in comparisons:
        if ref in deaths and exp in deaths and ref in births and exp in births:
            rate_exp = deaths[exp] / births[exp]
            rate_ref = deaths[ref] / births[ref]
            rr = rate_exp / rate_ref
            
            # Calcular o intervalo de confiança (IC 95%)
            ci_lower = rr * np.exp(-1.96 * np.sqrt((1/deaths[exp]) + (1/deaths[ref])))
            ci_upper = rr * np.exp(1.96 * np.sqrt((1/deaths[exp]) + (1/deaths[ref])))
            
            # Adicionar o resultado ao dicionário
            rr_results[f'{exp}_vs_{ref}'] = f'{round(rr, 2)} ({round(ci_lower, 2)}–{round(ci_upper, 2)})'
        else:
            # Se não houver dados suficientes para a comparação, retornar NaN
            rr_results[f'{exp}_vs_{ref}'] = 'NaN'
        
    rr_results['Prevalencia PIG IC 95%'] = prev_pig_ic
    rr_results['Prevalencia AIG IC 95%'] = prev_aig_ic
    rr_results['Prevalencia GIG IC 95%'] = prev_gig_ic
    return pd.DataFrame([rr_results])

# Exemplo de uso
# df_obito e df_nasc são os DataFrames de óbitos e nascimentos, respectivamente
# result = calculate_rr(df_obito, df_nasc, semana=24)
# print(result)

In [21]:
# df_obito e df_nasc são os DataFrames de óbitos e nascimentos, respectivamente
result = calculate_rr_e_prevalencia(df_obito=df_sim, df_nasc=df_sinasc, semana=24)
print(result)

   Gestational Week  Fetuses at Risk  Total Births  Live Births  Stillbirths  \
0                24         27235566         29793        20488         9305   

         PIG_vs_AIG        GIG_vs_AIG        PIG_vs_GIG  \
0  3.12 (2.98–3.27)  3.62 (3.44–3.81)  0.86 (0.82–0.91)   

  Prevalencia PIG IC 95% Prevalencia AIG IC 95% Prevalencia GIG IC 95%  
0     24.41 (23.92-24.9)     46.97 (46.4-47.53)    28.62 (28.11-29.14)  


# Brasil Total

In [ ]:
df_result_brasil = [calculate_rr_e_prevalencia(df_obito=df_sim, df_nasc=df_sinasc, semana=sem) for sem in range(22,43)]

In [ ]:
df_result_brasil = pd.concat(df_result_brasil)
df_result_brasil.to_csv('./resultado/brasil_total_por_semana_rr_e_prevalencia.csv', index=False)